## Package installation is required (only once)

In [ ]:
pip install -r packages.txt

In [2]:
import pybel
import pandas as pd
from pybel.dsl import Protein
from pybel.dsl import Abundance
from pybel.dsl import Pathology
from pybel.dsl import BiologicalProcess
from pybel.dsl import Population
from pybel.dsl import Gene
from pybel.dsl import Rna
from pybel.dsl import MicroRna
from pybel.dsl import Fragment
import chembl_webresource_client
import openpyxl
import networkx as nx
from pybel.io.jupyter import to_jupyter
import matplotlib.pyplot as plt
import chembl_webresource_client
from chembl_webresource_client.new_client import new_client
import pubchempy
import pickle
import re
#from ByCovidGraph import *
import pubchempy as pcp
from utils import *

In [9]:
infile = open('data/normalized_data/itmp_chem_dis','rb')
drugIndList = pickle.load(infile)
infile.close()

In [10]:
infile = open('data/normalized_data/itmp_chem_moa','rb')
mechList = pickle.load(infile)
infile.close()

In [11]:
infile = open('data/normalized_data/itmp_chem_act','rb')
ActList = pickle.load(infile)
infile.close()

In [12]:
infile = open('data/normalized_data/chem2Gene','rb')
chem2GeneList = pickle.load(infile)
infile.close()

In [13]:
infile = open('data/normalized_data/Chem2Gene2Path','rb')
chem2Gene_all = pickle.load(infile)
infile.close()

In [14]:
infile = open('data/normalized_data/ActList2gene2Path','rb')
ActList2gene2Path = pickle.load(infile)
infile.close()

In [15]:
%load ByCovidGraph.py

In [22]:
itmp_Graph = pybel.BELGraph(name='itmpGraph')

In [23]:
#convert chembl ids to proteins wherever possible
ActList2gene2Path = chembl2gene2path(chem2Gene_all,ActList)

In [24]:
#convert chembl ids to proteins wherever possible
mechList = chembl2gene2path(chem2Gene_all,mechList)

In [25]:
createGraph = chem2act_rel_2(ActList2gene2Path,itmp_Graph)
createGraph = chem2dis_rel(drugIndList,itmp_Graph)
createGraph = chem2moa_rel_2(mechList,itmp_Graph)
createGraph = chem2gene2path_rel(chem2Gene_all,itmp_Graph)

In [26]:
createGraph.summarize()

---------------------  ---------
Name                   itmpGraph
Version
Number of Nodes        13566
Number of Namespaces   6
Number of Edges        82970
Number of Annotations  0
Number of Citations    1
Number of Authors      0
Number of Components   30
Number of Warnings     0
Network Density        4.51E-04
---------------------  ---------

Type (6)             Count  Example
-----------------  -------  -----------------------------------------------------
Abundance             5045  a(ChEMBLAssay:CHEMBL3625892)
Pathology             3791  path(ChEMBL:CHEMBL2430359)
Population            1497  pop(Disease:"Pneumococcal Infections")
RNA                   1432  r(Pathway:"Apoptotic cleavage of cellular proteins.")
miRNA                 1062  m(HP:TST)
BiologicalProcess      739  bp(MOA:"Matrix metalloproteinase 12 inhibitor")

Prefix (6)    Name                        Count  Example
------------  ------------------------  -------  ------------------------------
ChEMBLAssay         

In [27]:
pybel.struct.summary.count_namespaces(createGraph)

Counter({'HP': 1062,
         'ChEMBLAssay': 5045,
         'ChEMBL': 3791,
         'Disease': 1497,
         'MOA': 739,
         'Pathway': 1432})

In [44]:
#Exporting the graph

#to cytoscape compatible graphml 
pybel.to_graphml(createGraph,'data/normalized_data/Full_itmpGraph.graphml')

#to regular BEL format
pybel.dump(createGraph,'data/normalized_data/Full_itmpGraph.bel')

#to neo4j
pybel.to_csv(createGraph,'data/normalized_data/Full_itmpGraph.csv')

#to sif
pybel.to_sif(createGraph,'data/normalized_data/Full_itmpGraph.sif')

## Visualize graph for top 51 ITMP compounds

In [29]:
itmp_top51 = pd.read_excel('data/normalized_data/CHEMBL_actives_Phenotypic.xlsx',sheet_name='Pchembl>6')

In [30]:
itmp_top51 = list(itmp_top51['Molecule ChEMBL ID'])
#type(itmp_top51)

In [31]:
itmp_top51[0]

'CHEMBL4303782'

In [33]:
nodes = [node for node in createGraph.nodes() if node.name in itmp_top51]
to_jupyter(pybel.struct.mutation.induction.get_subgraph_by_neighborhood(createGraph, nodes))

<IPython.core.display.Javascript object>

In [ ]:
#to_jupyter(pybel.struct.mutation.induction_expansion.get_subgraph_by_second_neighbors(test, nodes, filter_pathologies=False))

In [38]:
ActList2gene2Path['CHEMBL56731'][0]['Protein']

'EGFR'

# Monkeypox


In [94]:
# drugs from https://www.jglobalbiosecurity.com/articles/10.31646/gbio.12

mpox = 'CHEMBL152 CHEMBL203321 CHEMBL1257073 CHEMBL1242629'.split()
mpox

['CHEMBL152', 'CHEMBL203321', 'CHEMBL1257073', 'CHEMBL1242629']

In [4]:
getMech = RetMech(mpox)

[{'mechanism_of_action': 'Human herpesvirus 1 DNA polymerase inhibitor', 'target_chembl_id': 'CHEMBL1872'}]
[{'mechanism_of_action': 'DNA polymerase inhibitor', 'target_chembl_id': 'CHEMBL3988586'}]
[{'mechanism_of_action': 'Envelope phospholipase F13 (p37) inhibitor', 'target_chembl_id': 'CHEMBL4296170'}]
[]


In [21]:
getMech

{'CHEMBL152': [{'mechanism_of_action': 'Human herpesvirus 1 DNA polymerase inhibitor',
   'target_chembl_id': 'CHEMBL1872'}],
 'CHEMBL203321': [{'mechanism_of_action': 'DNA polymerase inhibitor',
   'target_chembl_id': 'CHEMBL3988586'}],
 'CHEMBL1257073': [{'mechanism_of_action': 'Envelope phospholipase F13 (p37) inhibitor',
   'target_chembl_id': 'CHEMBL4296170'}]}

In [5]:
getDis = RetDrugInd(mpox)

[{'mesh_heading': 'Virus Diseases'}, {'mesh_heading': 'Infections'}, {'mesh_heading': 'Kidney Diseases'}, {'mesh_heading': 'Sarcoma'}, '...(remaining elements truncated)...']
[{'mesh_heading': 'Cytomegalovirus Infections'}, {'mesh_heading': 'Hemorrhagic Fever, Ebola'}, {'mesh_heading': 'Adenovirus Infections, Human'}, {'mesh_heading': 'Adenoviridae Infections'}, '...(remaining elements truncated)...']
[{'mesh_heading': 'Virus Diseases'}, {'mesh_heading': 'Smallpox'}]
[]


In [9]:
getAct = RetAct(mpox,0)

1
[]
2
[]
3
[]
4
[]


In [10]:
getAct

{}

In [115]:
def RetAct(chemblIds,j):
    GetAct = new_client.activity
    ActList = []
    #for i in range(len(chemblIds)):
    #print(chemblIds[0])
    for i in range(len(chemblIds)):
        filtered_list=['assay_chembl_id','assay_type','pchembl_value','target_chembl_id','target_organism','bao_label']
        #acts = GetAct.filter(molecule_chembl_id=chemblIds[i],pchembl_value__isnull=False,assay_type_iregex='(B|F)',target_organism='Homo sapiens').only(filtered_list)
        #acts = GetAct.filter(molecule_chembl_id=chemblIds[i],pchembl_value__isnull=False,target_organism='Homo sapiens').only(filtered_list)
        acts = GetAct.filter(molecule_chembl_id=chemblIds[i],pchembl_value__isnull=False)
        j=j+1

        #acts = [d for d in acts if d.get('target_organism') == 'Homo sapiens']
        #acts = [d for d in acts if d.get('bao_label') == 'single protein format']
        #acts = [d for d in acts if d.get('type') in ['Ki', 'IC50']]
        acts = [d for d in acts if float(d.get('pchembl_value')) >= 6]
        print(j)
        #print(len(acts))
        acts = acts[:5]
        print(acts)
        ActList.append(list(acts))
    #print(ActList)
    named_ActList = dict(zip(chemblIds,ActList))
    named_ActList = {k: v for k, v in named_ActList.items() if v}
    return(named_ActList)


In [17]:
getAct = RetAct(mpox,0)

1
[{'activity_comment': None, 'activity_id': 302681, 'activity_properties': [], 'assay_chembl_id': 'CHEMBL746888', 'assay_description': 'Inhibitory activity against replication of HCMV in NHDF cells', 'assay_type': 'F', 'assay_variant_accession': None, 'assay_variant_mutation': None, 'bao_endpoint': 'BAO_0000190', 'bao_format': 'BAO_0000219', 'bao_label': 'cell-based format', 'canonical_smiles': 'Nc1ccn(C[C@@H](CO)OCP(=O)(O)O)c(=O)n1', 'data_validity_comment': None, 'data_validity_description': None, 'document_chembl_id': 'CHEMBL1129529', 'document_journal': 'J. Med. Chem.', 'document_year': 1996, 'ligand_efficiency': None, 'molecule_chembl_id': 'CHEMBL152', 'molecule_pref_name': 'CIDOFOVIR', 'parent_molecule_chembl_id': 'CHEMBL152', 'pchembl_value': '6.30', 'potential_duplicate': 0, 'qudt_units': 'http://www.openphacts.org/units/Nanomolar', 'record_id': 37715, 'relation': '=', 'src_id': 1, 'standard_flag': 1, 'standard_relation': '=', 'standard_text_value': None, 'standard_type': 'IC5

In [31]:
chemid = Ret_chembl_protein(getAct) + Ret_chembl_protein(getMech)
chemid

['CHEMBL613888',
 'CHEMBL613495',
 'CHEMBL613493',
 'CHEMBL613167',
 'CHEMBL614200',
 'CHEMBL613168',
 'CHEMBL612672',
 'CHEMBL371',
 'CHEMBL3988586',
 'CHEMBL4296170',
 'CHEMBL1872']

In [33]:
chem2uni = chembl2uniprot(chemid,0)

old 3
newLen 1
2
[{'component_synonym': 'POL', 'syn_type': 'GENE_SYMBOL'}]


In [34]:
chem2uni

{'CHEMBL3988586': [{'component_synonym': 'POL', 'syn_type': 'GENE_SYMBOL'}]}

In [36]:
getAct_new = chembl2gene2path(chem2uni,getAct)
getAct_new

{'CHEMBL152': [{'activity_comment': None,
   'activity_id': 302681,
   'activity_properties': [],
   'assay_chembl_id': 'CHEMBL746888',
   'assay_description': 'Inhibitory activity against replication of HCMV in NHDF cells',
   'assay_type': 'F',
   'assay_variant_accession': None,
   'assay_variant_mutation': None,
   'bao_endpoint': 'BAO_0000190',
   'bao_format': 'BAO_0000219',
   'bao_label': 'cell-based format',
   'canonical_smiles': 'Nc1ccn(C[C@@H](CO)OCP(=O)(O)O)c(=O)n1',
   'data_validity_comment': None,
   'data_validity_description': None,
   'document_chembl_id': 'CHEMBL1129529',
   'document_journal': 'J. Med. Chem.',
   'document_year': 1996,
   'ligand_efficiency': None,
   'molecule_chembl_id': 'CHEMBL152',
   'molecule_pref_name': 'CIDOFOVIR',
   'parent_molecule_chembl_id': 'CHEMBL152',
   'pchembl_value': '6.30',
   'potential_duplicate': 0,
   'qudt_units': 'http://www.openphacts.org/units/Nanomolar',
   'record_id': 37715,
   'relation': '=',
   'src_id': 1,
   'st

In [129]:
mpox_graph = pybel.BELGraph(name='mpox_chem')

In [40]:
mpox_graph = chem2moa_rel_2(getMech,mpox)
mpox_graph = chem2dis_rel(getDis,mpox)
mpox_graph = chem2act_rel_2(getAct,mpox)
#mpox_graph = chem2gene2path_rel(getAct,mpox)
to_jupyter(mpox_graph)

<IPython.core.display.Javascript object>

In [48]:
#reading file from pubchem for monkeypox 
#https://pubchem.ncbi.nlm.nih.gov/taxonomy/10244

mpox_pchem = pd.read_csv('data/normalized_data/TaxID_10244_bioactivity.csv')
mpox_pchem_cid = mpox_pchem['cid']
mpox_pchem_cid

0      11773602
1      11463022
2      16124688
3      16124688
4      16124688
        ...    
76     16124688
77    118895044
78    118895042
79    155545375
80    155521397
Name: cid, Length: 81, dtype: int64

In [54]:
len(set(mpox_pchem_cid))

24

In [53]:
cid2chembl_dict = []

for id in mpox_pchem_cid:
    #if id != 'None':
    print(id)
    GetChembl = {}
    c=pcp.Compound.from_cid(id)
    print(id)
    syn = c.synonyms
    for s in syn:
        #if s.startswith('SCHEMBL'):
            #GetChembl['SCHEMBL'] = s
        if s.startswith('CHEMBL'):
            GetChembl['CHEMBL'] = s

    cid2chembl_dict.append(GetChembl)
        

11773602
11773602
11463022
11463022
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
44420748
44420748
11243799
11243799
45111031
45111031
101879054
101879054
101879053
101879053
11452011
11452011
11301322
11301322
44341811
44341811
189926
189926
73124
73124
37542
37542
44341812
44341812
44341289
44341289
134406
134406
60613
60613
122873
122873
5271015
5271015
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16124688
16

In [66]:
list(cid2chembl_dict[0].values())

['CHEMBL220610']

In [80]:
chemblid = []

for id in cid2chembl_dict:
    #chemblid.append()
    #print(id.values())
    chembl = list(id.values())
    #print(chembl[0])
    chemblid.append(chembl[0])
    

In [100]:
chemblid = set(chemblid)
chemblid = list(chemblid) + mpox
chemblid

['CHEMBL220150',
 'CHEMBL113930',
 'CHEMBL321500',
 'CHEMBL114867',
 'CHEMBL219691',
 'CHEMBL221054',
 'CHEMBL3357215',
 'CHEMBL220098',
 'CHEMBL203321',
 'CHEMBL4565849',
 'CHEMBL3753489',
 'CHEMBL3753637',
 'CHEMBL1643',
 'CHEMBL3290663',
 'CHEMBL4449833',
 'CHEMBL1257073',
 'CHEMBL152',
 'CHEMBL392630',
 'CHEMBL494759',
 'CHEMBL326139',
 'CHEMBL112159',
 'CHEMBL360506',
 'CHEMBL3357216',
 'CHEMBL1242629',
 'CHEMBL375768',
 'CHEMBL220610',
 'CHEMBL152',
 'CHEMBL203321',
 'CHEMBL1257073',
 'CHEMBL1242629']

In [110]:
mpox

['CHEMBL152', 'CHEMBL203321', 'CHEMBL1257073', 'CHEMBL1242629']

In [120]:
pchem_mech = RetMech(chemblid)


[]
[]
[]
[]
[]
[]
[]
[]
[{'mechanism_of_action': 'DNA polymerase inhibitor', 'target_chembl_id': 'CHEMBL3988586'}]
[]
[]
[]
[{'mechanism_of_action': "Inosine-5'-monophosphate dehydrogenase 1 inhibitor", 'target_chembl_id': 'CHEMBL1822'}, {'mechanism_of_action': 'Hepatitis C virus NS5B RNA-dependent RNA polymerase inhibitor', 'target_chembl_id': 'CHEMBL5375'}, {'mechanism_of_action': 'RNA inhibitor', 'target_chembl_id': 'CHEMBL2363073'}]
[]
[]
[{'mechanism_of_action': 'Envelope phospholipase F13 (p37) inhibitor', 'target_chembl_id': 'CHEMBL4296170'}]
[{'mechanism_of_action': 'Human herpesvirus 1 DNA polymerase inhibitor', 'target_chembl_id': 'CHEMBL1872'}]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[{'mechanism_of_action': 'Human herpesvirus 1 DNA polymerase inhibitor', 'target_chembl_id': 'CHEMBL1872'}]
[{'mechanism_of_action': 'DNA polymerase inhibitor', 'target_chembl_id': 'CHEMBL3988586'}]
[{'mechanism_of_action': 'Envelope phospholipase F13 (p37) inhibitor', 'target_chembl_id': 'CHEMBL4296170'}]
[

In [121]:
pchem_mech

{'CHEMBL203321': [{'mechanism_of_action': 'DNA polymerase inhibitor',
   'target_chembl_id': 'CHEMBL3988586'}],
 'CHEMBL1643': [{'mechanism_of_action': "Inosine-5'-monophosphate dehydrogenase 1 inhibitor",
   'target_chembl_id': 'CHEMBL1822'},
  {'mechanism_of_action': 'Hepatitis C virus NS5B RNA-dependent RNA polymerase inhibitor',
   'target_chembl_id': 'CHEMBL5375'},
  {'mechanism_of_action': 'RNA inhibitor',
   'target_chembl_id': 'CHEMBL2363073'}],
 'CHEMBL1257073': [{'mechanism_of_action': 'Envelope phospholipase F13 (p37) inhibitor',
   'target_chembl_id': 'CHEMBL4296170'}],
 'CHEMBL152': [{'mechanism_of_action': 'Human herpesvirus 1 DNA polymerase inhibitor',
   'target_chembl_id': 'CHEMBL1872'}]}

In [122]:
pchem_dis = RetDrugInd(chemblid)

[]
[]
[]
[]
[]
[]
[]
[]
[{'mesh_heading': 'Cytomegalovirus Infections'}, {'mesh_heading': 'Hemorrhagic Fever, Ebola'}, {'mesh_heading': 'Adenovirus Infections, Human'}, {'mesh_heading': 'Adenoviridae Infections'}, '...(remaining elements truncated)...']
[]
[]
[]
[{'mesh_heading': 'Leukemia, Myeloid, Acute'}, {'mesh_heading': 'Hepatitis C, Chronic'}, {'mesh_heading': 'Hepatitis C'}, {'mesh_heading': 'Hepatitis B, Chronic'}, '...(remaining elements truncated)...']
[]
[]
[{'mesh_heading': 'Virus Diseases'}, {'mesh_heading': 'Smallpox'}]
[{'mesh_heading': 'Virus Diseases'}, {'mesh_heading': 'Infections'}, {'mesh_heading': 'Kidney Diseases'}, {'mesh_heading': 'Sarcoma'}, '...(remaining elements truncated)...']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[{'mesh_heading': 'Virus Diseases'}, {'mesh_heading': 'Infections'}, {'mesh_heading': 'Kidney Diseases'}, {'mesh_heading': 'Sarcoma'}, '...(remaining elements truncated)...']
[{'mesh_heading': 'Cytomegalovirus Infections'}, {'mesh_heading': 'Hemorrhagic Feve

In [123]:
pchem_dis

{'CHEMBL203321': [{'mesh_heading': 'Cytomegalovirus Infections'},
  {'mesh_heading': 'Hemorrhagic Fever, Ebola'},
  {'mesh_heading': 'Adenovirus Infections, Human'},
  {'mesh_heading': 'Adenoviridae Infections'},
  {'mesh_heading': 'Virus Diseases'},
  {'mesh_heading': 'Smallpox'}],
 'CHEMBL1643': [{'mesh_heading': 'Leukemia, Myeloid, Acute'},
  {'mesh_heading': 'Hepatitis C, Chronic'},
  {'mesh_heading': 'Hepatitis C'},
  {'mesh_heading': 'Hepatitis B, Chronic'},
  {'mesh_heading': 'Infections'},
  {'mesh_heading': 'Neoplasms'},
  {'mesh_heading': 'HIV Infections'},
  {'mesh_heading': 'Liver Cirrhosis'},
  {'mesh_heading': 'Hepatitis B, Chronic'},
  {'mesh_heading': 'Hemorrhagic Fever with Renal Syndrome'},
  {'mesh_heading': 'Hepatitis D'},
  {'mesh_heading': 'Encephalitis, Japanese'},
  {'mesh_heading': 'Respiratory Syncytial Virus Infections'},
  {'mesh_heading': 'Thrombocytopenia'},
  {'mesh_heading': 'Hepatitis'},
  {'mesh_heading': 'Virus Diseases'},
  {'mesh_heading': 'Fibrosis

In [124]:
pchem_act = RetAct(chemblid,0)

1
[{'activity_comment': None, 'activity_id': 1825368, 'activity_properties': [], 'assay_chembl_id': 'CHEMBL918571', 'assay_description': 'Antiviral activity against vaccinia virus IHD-J in BSC40 cells by CPE assay', 'assay_type': 'F', 'assay_variant_accession': None, 'assay_variant_mutation': None, 'bao_endpoint': 'BAO_0000188', 'bao_format': 'BAO_0000218', 'bao_label': 'organism-based format', 'canonical_smiles': 'O=C(NN1C(=O)[C@H]2C3C=CC(C4CC43)[C@H]2C1=O)c1ccncc1', 'data_validity_comment': None, 'data_validity_description': None, 'document_chembl_id': 'CHEMBL1142975', 'document_journal': 'J. Med. Chem.', 'document_year': 2007, 'ligand_efficiency': None, 'molecule_chembl_id': 'CHEMBL220150', 'molecule_pref_name': None, 'parent_molecule_chembl_id': 'CHEMBL220150', 'pchembl_value': '6.30', 'potential_duplicate': 0, 'qudt_units': 'http://www.openphacts.org/units/Nanomolar', 'record_id': 627667, 'relation': '=', 'src_id': 1, 'standard_flag': 1, 'standard_relation': '=', 'standard_text_va

7
[{'activity_comment': None, 'activity_id': 14986436, 'activity_properties': [], 'assay_chembl_id': 'CHEMBL3387285', 'assay_description': 'Inhibition of rabbit erythrocyte SAHase using S-adenosyl-L-homocysteine as substrate assessed as L-homocysteine formation incubated for 5 mins prior to substrate addition measured after 1 hr', 'assay_type': 'B', 'assay_variant_accession': None, 'assay_variant_mutation': None, 'bao_endpoint': 'BAO_0000190', 'bao_format': 'BAO_0000019', 'bao_label': 'assay format', 'canonical_smiles': 'Nc1ncc(F)c2c1ncn2[C@@H]1C[C@H](CO)[C@@H](O)[C@H]1O', 'data_validity_comment': None, 'data_validity_description': None, 'document_chembl_id': 'CHEMBL3351456', 'document_journal': 'Bioorg. Med. Chem.', 'document_year': 2014, 'ligand_efficiency': None, 'molecule_chembl_id': 'CHEMBL3357215', 'molecule_pref_name': None, 'parent_molecule_chembl_id': 'CHEMBL3357215', 'pchembl_value': '7.54', 'potential_duplicate': 0, 'qudt_units': 'http://www.openphacts.org/units/Nanomolar', 

10
[{'activity_comment': None, 'activity_id': 18969519, 'activity_properties': [], 'assay_chembl_id': 'CHEMBL4336406', 'assay_description': 'Antiviral activity against Ebolavirus Zaire infected in African green monkey Vero cells', 'assay_type': 'F', 'assay_variant_accession': None, 'assay_variant_mutation': None, 'bao_endpoint': 'BAO_0000188', 'bao_format': 'BAO_0000218', 'bao_label': 'organism-based format', 'canonical_smiles': 'Nc1nccc2c1ncn2C1=CC[C@@H](O)[C@H]1O', 'data_validity_comment': None, 'data_validity_description': None, 'document_chembl_id': 'CHEMBL4334462', 'document_journal': 'Bioorg Med Chem Lett', 'document_year': 2019, 'ligand_efficiency': None, 'molecule_chembl_id': 'CHEMBL4565849', 'molecule_pref_name': None, 'parent_molecule_chembl_id': 'CHEMBL4565849', 'pchembl_value': '6.37', 'potential_duplicate': 0, 'qudt_units': 'http://www.openphacts.org/units/Nanomolar', 'record_id': 3164411, 'relation': '=', 'src_id': 1, 'standard_flag': 1, 'standard_relation': '=', 'standar

14
[{'activity_comment': None, 'activity_id': 14710465, 'activity_properties': [], 'assay_chembl_id': 'CHEMBL3294635', 'assay_description': 'Inhibition of recombinant human AHCY using SAH as substrate assessed as formation of homocysteine after 10 mins', 'assay_type': 'B', 'assay_variant_accession': None, 'assay_variant_mutation': None, 'bao_endpoint': 'BAO_0000190', 'bao_format': 'BAO_0000357', 'bao_label': 'single protein format', 'canonical_smiles': 'Nc1ncc(F)c2c1ncn2[C@@H]1C[C@H](O)[C@@H](O)[C@H]1O', 'data_validity_comment': None, 'data_validity_description': None, 'document_chembl_id': 'CHEMBL3286270', 'document_journal': 'Bioorg. Med. Chem. Lett.', 'document_year': 2014, 'ligand_efficiency': {'bei': '28.41', 'le': '0.55', 'lle': '8.44', 'sei': '6.49'}, 'molecule_chembl_id': 'CHEMBL3290663', 'molecule_pref_name': None, 'parent_molecule_chembl_id': 'CHEMBL3290663', 'pchembl_value': '7.62', 'potential_duplicate': 0, 'qudt_units': 'http://www.openphacts.org/units/Nanomolar', 'record_

18
[{'activity_comment': None, 'activity_id': 6283281, 'activity_properties': [], 'assay_chembl_id': 'CHEMBL1803177', 'assay_description': 'Antiviral activity against Human cytomegalovirus Towne infected in HFF cells assessed as reduction in plaque formation after 10 days', 'assay_type': 'F', 'assay_variant_accession': None, 'assay_variant_mutation': None, 'bao_endpoint': 'BAO_0000190', 'bao_format': 'BAO_0000218', 'bao_label': 'organism-based format', 'canonical_smiles': 'Nc1ccn(C[C@H]2COP(=O)(O)CO2)c(=O)n1', 'data_validity_comment': None, 'data_validity_description': None, 'document_chembl_id': 'CHEMBL1799947', 'document_journal': 'Bioorg. Med. Chem. Lett.', 'document_year': 2011, 'ligand_efficiency': None, 'molecule_chembl_id': 'CHEMBL392630', 'molecule_pref_name': 'CYCLIC CIDOFOVIR', 'parent_molecule_chembl_id': 'CHEMBL392630', 'pchembl_value': '6.60', 'potential_duplicate': 0, 'qudt_units': 'http://www.openphacts.org/units/Nanomolar', 'record_id': 1442393, 'relation': '=', 'src_id

22
[{'activity_comment': None, 'activity_id': 13355179, 'activity_properties': [], 'assay_chembl_id': 'CHEMBL2400652', 'assay_description': 'Antiviral activity against Cowpox virus infected in african green monkey Vero 76 cells assessed as inhibition of virus-induced cytopathic effect after 3 days by microscopic analysis', 'assay_type': 'F', 'assay_variant_accession': None, 'assay_variant_mutation': None, 'bao_endpoint': 'BAO_0000190', 'bao_format': 'BAO_0000218', 'bao_label': 'organism-based format', 'canonical_smiles': 'Nc1ncnc2c1ncn2[C@@H]1C[C@H](CCO)[C@@H](O)[C@H]1O', 'data_validity_comment': None, 'data_validity_description': None, 'document_chembl_id': 'CHEMBL2396569', 'document_journal': 'Bioorg. Med. Chem.', 'document_year': 2013, 'ligand_efficiency': None, 'molecule_chembl_id': 'CHEMBL360506', 'molecule_pref_name': None, 'parent_molecule_chembl_id': 'CHEMBL360506', 'pchembl_value': '6.31', 'potential_duplicate': 0, 'qudt_units': 'http://www.openphacts.org/units/Nanomolar', 're

26
[{'activity_comment': None, 'activity_id': 1825362, 'activity_properties': [], 'assay_chembl_id': 'CHEMBL918571', 'assay_description': 'Antiviral activity against vaccinia virus IHD-J in BSC40 cells by CPE assay', 'assay_type': 'F', 'assay_variant_accession': None, 'assay_variant_mutation': None, 'bao_endpoint': 'BAO_0000188', 'bao_format': 'BAO_0000218', 'bao_label': 'organism-based format', 'canonical_smiles': 'O=C(NN1C(=O)[C@H]2C3C=CC(C4CC43)[C@H]2C1=O)c1ccc(Br)cc1', 'data_validity_comment': None, 'data_validity_description': None, 'document_chembl_id': 'CHEMBL1142975', 'document_journal': 'J. Med. Chem.', 'document_year': 2007, 'ligand_efficiency': None, 'molecule_chembl_id': 'CHEMBL220610', 'molecule_pref_name': None, 'parent_molecule_chembl_id': 'CHEMBL220610', 'pchembl_value': '7.70', 'potential_duplicate': 0, 'qudt_units': 'http://www.openphacts.org/units/Nanomolar', 'record_id': 627673, 'relation': '=', 'src_id': 1, 'standard_flag': 1, 'standard_relation': '=', 'standard_te

In [125]:
chemid2 = Ret_chembl_protein(pchem_act) + Ret_chembl_protein(pchem_mech)
chemid2

['CHEMBL1293235',
 'CHEMBL276',
 'CHEMBL613495',
 'CHEMBL2664',
 'CHEMBL613493',
 'CHEMBL613168',
 'CHEMBL612558',
 'CHEMBL613494',
 'CHEMBL340',
 'CHEMBL614200',
 'CHEMBL612672',
 'CHEMBL371',
 'CHEMBL4434621',
 'CHEMBL3879801',
 'CHEMBL613167',
 'CHEMBL3751646',
 'CHEMBL613120',
 'CHEMBL612545',
 'CHEMBL613888',
 'CHEMBL613332',
 'CHEMBL613738',
 'CHEMBL614149',
 'CHEMBL4296170',
 'CHEMBL3988586',
 'CHEMBL1872',
 'CHEMBL2363073',
 'CHEMBL1822',
 'CHEMBL5375']

In [126]:
chem2uni = chembl2uniprot(chemid2,0)
chem2uni

old 10
newLen 8
2
[{'xref_id': 'R-HSA-1221632', 'xref_name': 'Meiotic synapsis. [P02545-2]', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-2980766', 'xref_name': 'Nuclear Envelope Breakdown.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-2995383', 'xref_name': 'Initiation of Nuclear Envelope (NE) Reformation.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-352238', 'xref_name': 'Breakdown of the nuclear lamina. [P02545-1]', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-381038', 'xref_name': 'XBP1(S) activates chaperone genes.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-4419969', 'xref_name': 'Depolymerisation of the Nuclear Lamina.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-6802952', 'xref_name': 'Signaling by BRAF and RAF fusions.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-8862803', 'xref_name': "Deregulated CDK5 triggers multiple neurodegenerative pathways in Alzheimer's disease models. [P02545-1]", 'xref_src_db': 'Reactome'}, {'component_synonym': 'LMNA', 'syn_

{'CHEMBL1293235': [{'xref_id': 'R-HSA-1221632',
   'xref_name': 'Meiotic synapsis. [P02545-2]',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-2980766',
   'xref_name': 'Nuclear Envelope Breakdown.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-2995383',
   'xref_name': 'Initiation of Nuclear Envelope (NE) Reformation.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-352238',
   'xref_name': 'Breakdown of the nuclear lamina. [P02545-1]',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-381038',
   'xref_name': 'XBP1(S) activates chaperone genes.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-4419969',
   'xref_name': 'Depolymerisation of the Nuclear Lamina.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-6802952',
   'xref_name': 'Signaling by BRAF and RAF fusions.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-8862803',
   'xref_name': "Deregulated CDK5 triggers multiple neurodegenerative pathways in Alzheimer's disease models. [P02545-1]",
   '

In [128]:
pchem_act_new= chembl2gene2path(chem2uni,pchem_act)
pchem_act_new

{'CHEMBL220150': [{'activity_comment': None,
   'activity_id': 1825368,
   'activity_properties': [],
   'assay_chembl_id': 'CHEMBL918571',
   'assay_description': 'Antiviral activity against vaccinia virus IHD-J in BSC40 cells by CPE assay',
   'assay_type': 'F',
   'assay_variant_accession': None,
   'assay_variant_mutation': None,
   'bao_endpoint': 'BAO_0000188',
   'bao_format': 'BAO_0000218',
   'bao_label': 'organism-based format',
   'canonical_smiles': 'O=C(NN1C(=O)[C@H]2C3C=CC(C4CC43)[C@H]2C1=O)c1ccncc1',
   'data_validity_comment': None,
   'data_validity_description': None,
   'document_chembl_id': 'CHEMBL1142975',
   'document_journal': 'J. Med. Chem.',
   'document_year': 2007,
   'ligand_efficiency': None,
   'molecule_chembl_id': 'CHEMBL220150',
   'molecule_pref_name': None,
   'parent_molecule_chembl_id': 'CHEMBL220150',
   'pchembl_value': '6.30',
   'potential_duplicate': 0,
   'qudt_units': 'http://www.openphacts.org/units/Nanomolar',
   'record_id': 627667,
   're

In [133]:
mpox_graph = chem2moa_rel_2(pchem_mech,mpox_graph)
mpox_graph = chem2dis_rel(pchem_dis,mpox_graph)
mpox_graph = chem2act_rel_2(pchem_act_new,mpox_graph)
mpox_graph = chem2gene2path_rel(chem2uni,mpox_graph)
to_jupyter(mpox_graph)

<IPython.core.display.Javascript object>